In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
from tqdm import tqdm
from utils.evaluation import gurobi_solve_lp

import os
import torch
from scipy.linalg import LinAlgError
import numpy as np
from torch_geometric.data import Batch, HeteroData
from scipy.sparse import coo_array

from generate_instances_lp import generate_setcover, Graph, generate_indset, generate_cauctions, generate_capacited_facility_location
from utils.evaluation import data_contraint_heuristic, data_inactive_constraints, normalize_cons

In [ ]:
rng = np.random.RandomState(1)

In [ ]:
root = 'datasets/setc_100_100_0.05'
os.mkdir(root)
os.mkdir(os.path.join(root, 'processed'))

### Setcover

In [ ]:
def surrogate_gen():
    nrows = 100
    ncols = 100
    density = 0.05
    nnzrs = int(nrows * ncols * density)
    A, b, c = generate_setcover(nrows, ncols, nnzrs, rng)
    return A, b, c

### Indset

In [ ]:
def surrogate_gen():
    nnodes = 100
    edge_probability = 0.02
    graph = Graph.erdos_renyi(number_of_nodes=nnodes, edge_probability=edge_probability, random=rng)
    A, b, c = generate_indset(graph=graph, nnodes=nnodes)
    return A, b, c

### Cauctions

In [ ]:
def surrogate_gen():
    n_items=100
    n_bids=100
    A, b, c = generate_cauctions(n_items=n_items, n_bids=n_bids, rng=rng, min_value=0.5, max_value=1., add_item_prob=0.5)
    # c = np.ones_like(c, dtype=np.float32) * -1.
    return A, b, c

### Facilities

In [ ]:
def surrogate_gen():
    n_customers = 25
    n_facilities = 3
    ratio = 0.5
    # min would be like 0.2-ish
    A_ub, b_ub, c = generate_capacited_facility_location(n_customers=n_customers, 
                                                                     n_facilities=n_facilities, 
                                                                     ratio=ratio, rng=rng)
    return A_ub, b_ub, c

# create ineq

In [ ]:
ips = []
graphs = []
pkg_idx = 0
success_cnt = 0

max_iter = 15000
num = 10000

pbar = tqdm(range(max_iter))
for i in pbar:
    A, b, c = surrogate_gen()
    c = c / (np.abs(c).max() + 1.e-10)  # does not change the result
    A, b = normalize_cons(A, b)
    
    try:
        solution, duals = gurobi_solve_lp(A, b, c)
        assert solution is not None
        assert c.dot(solution) != 0.
    except (AssertionError, LinAlgError):
        continue
    else:
        # heur_idx = data_contraint_heuristic(None, A, b, c)
        inactive_idx = data_inactive_constraints(A, b, solution)
        # inactive_heur_acc = np.isin(heur_idx, inactive_idx).sum() / len(heur_idx)

        A = torch.from_numpy(A).to(torch.float)
        b = torch.from_numpy(b).to(torch.float)
        c = torch.from_numpy(c).to(torch.float)
        x = torch.from_numpy(solution).to(torch.float)

        A_where = torch.where(A)
        data = HeteroData(
            cons={
                'num_nodes': b.shape[0],
                'x': torch.empty(b.shape[0]),
                 },
            vals={
                'num_nodes': c.shape[0],
                'x': torch.empty(c.shape[0]),
            },
            cons__to__vals={'edge_index': torch.vstack(A_where),
                            'edge_attr': A[A_where][:, None]},
            x_solution=x,
            duals=torch.from_numpy(duals).float(),
            obj_solution=c.dot(x),
            q=c,
            b=b,
            # heur_idx=torch.from_numpy(heur_idx).long(),
            inactive_idx=torch.from_numpy(inactive_idx).long(),
        )
        success_cnt += 1
        graphs.append(data)

    if len(graphs) >= 1000 or success_cnt == num:
        torch.save(Batch.from_data_list(graphs), f'{root}/processed/batch{pkg_idx}.pt')
        pkg_idx += 1
        graphs = []

    if success_cnt >= num:
        break

    pbar.set_postfix({'suc': success_cnt, 'obj': c.numpy().dot(solution)})

In [ ]:
from data.dataset import LPDataset

In [ ]:
ds = LPDataset(root, 'test')

In [ ]:
data = ds[0]